In [1]:
import paho.mqtt.client as mqtt
import paho.mqtt.publish as publish

from pymongo import MongoClient
from bson.objectid import ObjectId

import time,datetime
import os 
import schedule

import pandas as pd
import numpy as np

import math

import matplotlib.pyplot as plt
from matplotlib import style

from copy import deepcopy
from IPython.display import clear_output

In [7]:
# def realtimetotalstep():
conn = MongoClient('120.126.136.17')
db = conn.Tracker
collection = [db.rank]# db.db2, db.dn2, db.james, db.leo
for col in collection:
    cursor = col.find({})
    df = pd.DataFrame(list(cursor))
    
df

,_id,day,hour,james,jeng,leo,month,shuan,t,user,year
0,5cd2af46c4e5a62a84bae940,8,18.0,-1719.0,2907.0,0.0,5,3850.0,3850.0,leo,2019
1,5cd2af46c4e5a62a84bae941,8,18.0,0.0,4626.0,1719.0,5,5569.0,5569.0,james,2019
2,5cd2af46c4e5a62a84bae942,8,18.0,-5569.0,-943.0,-3850.0,5,0.0,0.0,shuan,2019
3,5cd2af46c4e5a62a84bae943,8,18.0,-5569.0,-943.0,-3850.0,5,0.0,0.0,t,2019
4,5cd2af46c4e5a62a84bae944,8,18.0,-4626.0,0.0,-2907.0,5,943.0,943.0,jeng,2019
5,5cd2b7ddc4e5a65296bb45f2,8,19.0,-1719.0,2907.0,0.0,5,3850.0,3850.0,leo,2019
6,5cd2b7ddc4e5a65296bb45f3,8,19.0,0.0,4626.0,1719.0,5,5569.0,5569.0,james,2019
7,5cd2b7ddc4e5a65296bb45f4,8,19.0,-5569.0,-943.0,-3850.0,5,0.0,0.0,shuan,2019
8,5cd2b7ddc4e5a65296bb45f5,8,19.0,-5569.0,-943.0,-3850.0,5,0.0,0.0,t,2019
9,5cd2b7ddc4e5a65296bb45f6,8,19.0,-4626.0,0.0,-2907.0,5,943.0,943.0,jeng,2019
